In [8]:
from MultiTrackingSystem import MultiTrackingSystem
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import matplotlib.animation as animation
import cv2

import open3d as op3

ModuleNotFoundError: No module named 'open3d'

In [ ]:
alpha = np.pi * (0.2)/180
beta = 12
corr = np.sin(alpha/2) * 2
min_sample_1 = 19657
min_sample_2 = -2.138
min_sample_3 = -100

In [ ]:
os.chdir(r'/Users/czhui960/Documents/Lidar/to ZHIHUI/USA pkwy')
file_path  = os.listdir()[4]
multi_tracking = MultiTrackingSystem(iter_n = 1800, tolerance= 4,gen_fig= True)
multi_tracking.fit_adbgen_pcap(file_path,beta,min_sample_1,min_sample_2,min_sample_3)
multi_tracking.batch_tracking()
multi_tracking.svae_gif('USA pkwy')

In [ ]:
os.chdir(r'/Users/czhui960/Documents/Lidar/to ZHIHUI/USA pkwy')
file_path  = os.listdir()[4]
multi_tracking = MultiTrackingSystem(iter_n = 100, tolerance= 4,gen_fig= False)
multi_tracking.fit_adbgen_pcap(file_path,beta,min_sample_1,min_sample_2,min_sample_3)
multi_tracking.batch_tracking()

In [ ]:
def extract_pcds(tracking_list,obj_id):
    pcds = []
    for i in range(len(tracking_list[obj_id].point_clouds)):
        if type(tracking_list[obj_id].point_clouds[i]) != int:
            xy = tracking_list[obj_id].point_clouds[i]
            z = tracking_list[obj_id].elevation_intensities[i][:,0].reshape(-1,1)
            xyz = np.concatenate([xy,z],axis = 1)
            intensity = tracking_list[obj_id].elevation_intensities[i][:,1]
            pcd = op3.geometry.PointCloud()
            pcd.points = op3.utility.Vector3dVector(xyz)
            pcds.append(pcd)
    return pcds

In [ ]:
def show_three_frame_traj(pcds,ind):
    start = ind - 1 
    end = ind + 1 
    if start<0:
        start = 0
    if end > (len(pcds)-1):
        end = len(pcds) - 1
    target = pcds[ind]
    pcl,_ = target.compute_convex_hull()# convex hull
    hull_ls = op3.geometry.LineSet.create_from_triangle_mesh(pcl)
    hull_ls.paint_uniform_color((1,0,0))
    op3.visualization.draw_geometries([target,hull_ls,pcds[start],pcds[end]])

In [ ]:
def show_all_traj(pcds):
    op3.visualization.draw_geometries(pcds)

In [ ]:
def is_input_correct(cur_input):
    if cur_input == 'y' or cur_input == 'Y' or cur_input == 'n' or cur_input == 'N' or cur_input == 'exit':
        return True
    else:
        return False

In [ ]:
cur_object_ind = 0 
processed_point_clouds_num = 0
comfirmed_samples = []
labels = [] # 0: car, 1: horse, 2:noise
while True:
    pcds = extract_pcds(multi_tracking.out_of_tracking_list,cur_object_ind)
    show_all_traj(pcds)
    cur_input = input('Save All Trackings?:')
    correct_input = is_input_correct(cur_input)
    if cur_input == 'exit':
        break
    while correct_input:
        correct_input = is_input_correct(cur_input)
        print('Wrong Input')
        if correct_input:
            break
    if cur_input == 'y' or cur_input == 'Y':
        added_samples = 0
        for i in range(len(pcds)):
            comfirmed_samples.append(pcds[i])
            added_samples += 1
        processed_point_clouds_num += added_samples
    else:
        for i in range(len(pcds)):
            show_three_frame_traj(pcds,i)
            cur_input = input('Save the Labelled Tracking?:')
            while correct_input:
                correct_input = is_input_correct(cur_input)
                print('Wrong Input')
                if correct_input:
                    break
            if cur_input == 'exit':
                break
            if cur_input == 'y' or cur_input == 'Y':
                comfirmed_samples.append(pcd[i])
                processed_point_clouds_num += 1
    cur_object_ind+=1

                
                


        